In [1]:
import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt
import csv
import math


#ピクセル数をmmに変換するー
path='C:\\Users\\flow\\Desktop\\2022.2.22ctab\\2022.2.22ctab1\\'
filename="C:\\Users\\flow\\Desktop\\輪郭検出データ\\2022.2.22 ctab長さ.mp4"
cap = cv2.VideoCapture(filename=filename)
num = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        if num<3:
            mm_img=cv2.imwrite(path+"mmpicture{:0=3}".format(num)+".bmp",frame)
            #print("save mmpicture{:0=3}".format(num)+".bmp")
            num+=1
    else:
        break  
cap.release() 

#画像クリックしたところの座標取得
#画像の上下端の座標とる際に同じx座標の位置をプロットしないと正しい距離が出せないから注意!!
file_name=path+'mmpicture001.bmp'
img=cv2.imread(file_name,cv2.IMREAD_GRAYSCALE)
global x
global y
xlist=[]
ylist=[]
def click_pos(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        img2 = np.copy(img)
        cv2.circle(img2,center=(x,y),radius=5,color=255,thickness=-1)
        pos_str='(x,y)=('+str(x)+','+str(y)+')'
        cv2.putText(img2,pos_str,(x+10, y+10),cv2.FONT_HERSHEY_PLAIN,2,255,2,cv2.LINE_AA)
        cv2.imshow('window', img2)
        #print(x,y)
        xlist.append(x)
        ylist.append(y)
        
cv2.imshow('window', img)
cv2.setMouseCallback('window', click_pos)
cv2.waitKey(0)
cv2.destroyAllWindows() 
print(xlist)
print(ylist)
dis_list=[]
for a in range(len(xlist)-1):
    x1=xlist[a]
    x2=xlist[a+1]
    y1=ylist[a]
    y2=ylist[a+1]
    dis=((x1-x2)**2+(y1-y2)**2)**(1/2)
    dis_list.append(dis)
    #print(x1,y1,x2,y2)
    #print(dis)
mean=sum(dis_list)/float(len(dis_list))
print(mean)
mm=mean/10
print('ピクセル数/mm=',mm,'p/mm')
list_mm=['ピクセル数/mm=',mm,'p/mm']
#--------------------------------------------------------------------------------------------------------------------------
#csvファイルに保存してくよ
import csv
with open(path+'mmchange.csv','w',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(list_mm)


[725, 1342, 722, 1339]
[472, 450, 475, 456]
618.3961328772576
ピクセル数/mm= 61.83961328772576 p/mm


In [9]:
###輪郭検出####

# ビデオデータの読み込み
path1='C:\\Users\\flow\\Desktop\\2022.2.22ctab\\2022.2.22ctab2\\'
filename ="C:\\Users\\flow\\Desktop\\輪郭検出データ\\2022.2.22 ctab膨張.mp4"
# 動画の読み込み
cap = cv2.VideoCapture(filename=filename)
#動画ファイルの情報を取得する
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
hight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
total_frame = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)

#print("動画の幅: ", width)
#print("動画の高さ: ", hight)
print("総フレーム数: ", total_frame)
print("fps: ", fps)
#print("動画の長さ: ", total_frame/fps, "[sec]")

#print(type(cap))
#print(cap.isOpened())
#------------------------------------------------------------------------------------------------------------------
num = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        if num<400:
        
            img=cv2.imwrite(path1+"picture{:0=3}".format(num)+".bmp",frame)
            #print("save picture{:0=3}".format(num)+".bmp")
            img1=cv2.imread(path1+"picture{:0=3}".format(num)+".bmp")
            #print(type(img1))
            img2=cv2.rotate(img1,cv2.ROTATE_90_COUNTERCLOCKWISE)
            kaiten_img=cv2.imwrite(path1+"kaitenpicture{:0=3}".format(num)+".bmp",img2)
            #print("save kaitenpicture{:0=3}".format(num)+".bmp")
            num += 1
        
    else:
        break

cap.release()
#--------------------------------------------------------------------------------------------------------------
# 静止画像の抜き出し
# 入力画像の読み込み
i=-30
j=0
idx_max_list=[]
idx_min_list=[]
for a in range(int(num/30)):
    i+=30
    j+=30
    #print('フレーム範囲',i,j)
    Area_list=[]
    for b in range(i,j):    

        img3 = cv2.imread(path1+"kaitenpicture{:0=3}".format(b)+".bmp")
        
        #グレースケールに変換
        gray = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)    
        # グレースケール画像の書き込み
        cv2.imwrite(path1+"graypicture{:0=3}".format(b)+".bmp", gray)
        #print("save graypicture{:0=3}".format(b)+".bmp")
    #---------------------------------------------------------------------------------------------------------------  
        # 2値化する。
        #ret, binary = cv2.threshold(gray,70, 255, cv2.THRESH_BINARY_INV)
        ret, binary = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        cv2.imwrite(path1+"nichipicture{:0=3}".format(b)+".bmp",binary)

        #見つけた輪郭を元画像に描画
        contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        img4=cv2.imread(path1+"kaitenpicture{:0=3}".format(b)+".bmp")
        #print(cv2.contourArea(contours[len(contours)-1]))
        for k in range(len(contours)):
                #最上層の階層を描画（緑）
            if hierarchy[0][k][3] ==-1:        
                cv2.drawContours(img4, contours, k, (0, 255, 0), 2)
            #2階層目を描画（水色）
            else:
                cv2.drawContours(img4, contours, k, (0, 255, 255), 2)
        cv2.imwrite(path1+"rinkakupicture{:0=3}".format(b)+".bmp",img4)
        #print("save rinkakupicture{:0=3}".format(b)+".bmp")
    #----------------------------------------------------------------------------------------
        #気泡の面積出して膨張・収縮判断する
        MAX_list= max(contours, key=lambda x: cv2.contourArea(x))
        Area= cv2.contourArea(MAX_list)
        #print('area{:0=3}'.format(b),Area)

    #面積をリストに格納からの面積の大きさ可視化
        Area_list.append(Area)
    #print(Area_list)
    max_Area=max(Area_list)
    min_Area=min(Area_list)
    #print(max_Area)    
    #print(min_Area)        
    max_num=Area_list.index(max_Area)
    min_num=Area_list.index(min_Area) 
    #print('膨張だお',max_num)
    #print('収縮だお',min_num)

    #写真の本当の番号指定
    idx_max=max_num+30*a
    idx_min=min_num+30*a
    #print('膨張idx',idx_max)
    #print('収縮idx',idx_min)
    idx_max_list.append(idx_max)
    idx_min_list.append(idx_min)

#-------------------------------------------------------------------------------------------------------------------------------  
    #対応する画像保存だよー
    path2='C:\\Users\\flow\\Desktop\\2022.2.22ctab\\2022.2.22ctab3\\'
    puffed_img=cv2.imread(path1+"kaitenpicture{:0=3}".format(idx_max)+".bmp")  
    shrink_img=cv2.imread(path1+"kaitenpicture{:0=3}".format(idx_min)+".bmp")
    cv2.imwrite(path2+"bouchoupicture{:0=3}".format(a)+".bmp",puffed_img)
    cv2.imwrite(path2+"shushukupicture{:0=3}".format(a)+".bmp",shrink_img)  
#---------------------------------------------------------------------------------------
#print(idx_max_list)
#print(idx_min_list)


総フレーム数:  645.0
fps:  29.97002997002997


In [8]:
#膨張・収縮時で速度出すよ
#膨張バージョン
n=-1
m=1
#idx_max_list
u_list=[]
s_list=[]
y_list=[]
#print(idx_max_list)
for c in range(len(idx_max_list)-1):
        n+=1
        m=n+1
        #print('範囲',n,m)
        frame1=cv2.imread(path2+"bouchoupicture{:0=3}".format(n)+".bmp")
        frame2=cv2.imread(path2+"bouchoupicture{:0=3}".format(m)+".bmp")
        #print("save frame1picture{:0=3}".format(n)+".bmp")
        #print("save frame2picture{:0=3}".format(m)+".bmp")
        gray1=cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
        gray2=cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
        ret,th_otsu1 = cv2.threshold(gray1,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        ret,th_otsu2 = cv2.threshold(gray2,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

        #モーメント関数で3次までのモーメントを取得
        mu1 = cv2.moments(th_otsu1)
        mu2 = cv2.moments(th_otsu2)
        #図心を出す
        x1,y1= int(mu1["m10"]/mu1["m00"]) , int(mu1["m01"]/mu1["m00"])
        x2,y2= int(mu2["m10"]/mu2["m00"]) , int(mu2["m01"]/mu2["m00"])
        y_list.append(y1)
        y_list.append(y2)
        #図心を画像に記載する
        cv2.circle(frame1,(x1,y1),20,(0,0,255),-1)
        if c==len(idx_max_list)-1:
                cv2.circle(frame2,(x2,y2),20,(0,0,255),-1)
                cv2.imwrite(path2+'bouchoframe_moment{:0=3}'.format(m)+".bmp",frame2)
        else:
        #画像を表示する
                cv2.imwrite(path2+'bouchoframe_moment{:0=3}'.format(n)+".bmp",frame1)
        #plt.imshow(frame)
        #plt.show
        #print(y1,y2)
        
#----------------------------------------------------------------
#速度出すよ
        y_ex=y2-y1
        idx_y1=idx_max_list[n]
        idx_y2=idx_max_list[m]
        s_ex=(idx_y2-idx_y1)/30
        #print(idx_y1,idx_y2)
        v_ex=y_ex/s_ex
        #print('速度',v,'p/s')
#p/sをmm/sに変換してくよ
        u_ex=v_ex/mm
        u_list.append(u_ex)
        s_list.append(s_ex)
        print('膨張時上昇速度',u_ex,'mm/s')
#print(y_list)
        
#収縮時バージョン
n=-1
m=1
#idx_min_list
uuu_list=[]
sss_list=[]
yyy_list=[]
#print(idx_min_list)
for f in range(len(idx_min_list)-1):
        n+=1
        m=n+1
        #print('範囲',n,m)
        frame1=cv2.imread(path2+"shushukupicture{:0=3}".format(n)+".bmp")
        frame2=cv2.imread(path2+"shushukupicture{:0=3}".format(m)+".bmp")
        #print("save frame1picture{:0=3}".format(n)+".bmp")
        #print("save frame2picture{:0=3}".format(m)+".bmp")
        gray1=cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
        gray2=cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
        ret,th_otsu1 = cv2.threshold(gray1,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        ret,th_otsu2 = cv2.threshold(gray2,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

        #モーメント関数で3次までのモーメントを取得
        mu1 = cv2.moments(th_otsu1)
        mu2 = cv2.moments(th_otsu2)
        #図心を出す
        x1,y1= int(mu1["m10"]/mu1["m00"]) , int(mu1["m01"]/mu1["m00"])
        x2,y2= int(mu2["m10"]/mu2["m00"]) , int(mu2["m01"]/mu2["m00"])
        yyy_list.append(y1)
        yyy_list.append(y2)
        #図心を画像に記載する
        cv2.circle(frame1,(x1,y1),20,(0,0,255),-1)
        if c==len(idx_min_list)-1:
                cv2.circle(frame2,(x2,y2),20,(0,0,255),-1)
                cv2.imwrite(path2+'shushukuframe_moment{:0=3}'.format(m)+".bmp",frame2)
        else:
        #画像を表示する
                cv2.imwrite(path2+'shushukuframe_moment{:0=3}'.format(n)+".bmp",frame1)
        #plt.imshow(frame)
        #plt.show
        #print(y1,y2)
#----------------------------------------------------------------
#速度出すよ
        y_con=y2-y1
        idx_y1=idx_min_list[n]
        idx_y2=idx_min_list[m]
        s_con=(idx_y2-idx_y1)/30
        #print(idx_y1,idx_y2)
        v_con=y_con/s_con
        #print('速度',v,'p/s')
#p/sをmm/sに変換してくよ
        u_con=v_con/mm
        uuu_list.append(u_con)
        sss_list.append(s_con)
        print('収縮時上昇速度',u_con,'mm/s')        


膨張時上昇速度 0.11319605068405877 mm/s
膨張時上昇速度 0.250927205949884 mm/s
膨張時上昇速度 0.015649223596874484 mm/s
膨張時上昇速度 0.20074176475990718 mm/s
膨張時上昇速度 0.0970251863006218 mm/s
収縮時上昇速度 0.08085432191718483 mm/s
収縮時上昇速度 0.11709936277661252 mm/s
収縮時上昇速度 0.06468345753374788 mm/s
収縮時上昇速度 0.08085432191718483 mm/s
収縮時上昇速度 0.06468345753374788 mm/s


In [5]:
#自然上昇時の速度
filename ="C:\\Users\\flow\\Desktop\\輪郭検出データ\\2022.2.22 ctab自然.mp4\\"
cap = cv2.VideoCapture(filename=filename)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
hight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
total_frame = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)
#print("動画の幅: ", width)
#print("動画の高さ: ", hight)
#print("総フレーム数: ", total_frame)
#print("fps: ", fps)
#print("動画の長さ: ", total_frame/fps, "[sec]")
path3='C:\\Users\\flow\\Desktop\\2022.2.22ctab\\2022.2.22ctab natural\\'

num = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        if num<200:
            natural_img=cv2.imwrite(path3+"picture{:0=3}".format(num)+".bmp",frame)
            #print("save picture{:0=3}".format(num)+".bmp")
            natural_img1=cv2.imread(path3+"picture{:0=3}".format(num)+".bmp")
            #print(type(img1))
            natural_img2=cv2.rotate(natural_img1,cv2.ROTATE_90_COUNTERCLOCKWISE)
            kaiten_img=cv2.imwrite(path3+"kaitenpicture{:0=3}".format(num)+".bmp",natural_img2)
            #print("save kaitenpicture{:0=3}".format(num)+".bmp")
            num+=1
    else:
        break  
cap.release() 
#--------------------------------------------------------------------------------------------------------------
#膨張時の画像と対応する画像の抜き出しする
#print(idx_max_list)
for idx in idx_max_list:
    natural_img3=cv2.imread(path3+"kaitenpicture{:0=3}".format(idx)+".bmp")  
    #グレースケールに変換
    gray = cv2.cvtColor(natural_img3, cv2.COLOR_BGR2GRAY) 
    # グレースケール画像の書き込み
    cv2.imwrite(path3+"graypicture{:0=3}".format(idx)+".bmp", gray)
    #print("save graypicture{:0=3}".format(idx)+".bmp")
    #---------------------------------------------------------------------------------------------------------------  
    # 2値化する。
    #ret, binary = cv2.threshold(gray,70, 255, cv2.THRESH_BINARY_INV)
    ret, binary = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    cv2.imwrite(path3+"nichipicture{:0=3}".format(idx)+".bmp",binary)
    #見つけた輪郭を元画像に描画
    contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    natural_img4=cv2.imread(path3+"kaitenpicture{:0=3}".format(idx)+".bmp")
    #print(cv2.contourArea(contours[len(contours)-1]))
    for k in range(len(contours)):
            #最上層の階層を描画（緑）
        if hierarchy[0][k][3] ==-1:        
            cv2.drawContours(natural_img4, contours, k, (0, 255, 0), 2)
        #2階層目を描画（水色）
        else:
            cv2.drawContours(natural_img4, contours, k, (0, 255, 255), 2)
    cv2.imwrite(path3+"rinkakupicture{:0=3}".format(idx)+".bmp",natural_img4)
    #print("save rinkakupicture{:0=3}".format(idx)+".bmp")
#--------------------------------------------------------------------
#上昇速度出すんご
uu_list=[]
ss_list=[]
yy_list=[]
aa=-1

for taishi in range(len(idx_max_list)-1):
    aa+=1
    n=idx_max_list[aa]
    m=idx_max_list[aa+1]
    #print('範囲',n,m)    

    frame1=cv2.imread(path3+"kaitenpicture{:0=3}".format(n)+".bmp")
    frame2=cv2.imread(path3+"kaitenpicture{:0=3}".format(m)+".bmp")
    #print("save frame1picture{:0=3}".format(n)+".bmp")
    #print("save frame2picture{:0=3}".format(m)+".bmp")
    gray1=cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    gray2=cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    ret,th_otsu1 = cv2.threshold(gray1,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    ret,th_otsu2 = cv2.threshold(gray2,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    #モーメント関数で3次までのモーメントを取得
    mu1 = cv2.moments(th_otsu1)
    mu2 = cv2.moments(th_otsu2)
    #図心を出す
    x1,y1= int(mu1["m10"]/mu1["m00"]) , int(mu1["m01"]/mu1["m00"])
    x2,y2= int(mu2["m10"]/mu2["m00"]) , int(mu2["m01"]/mu2["m00"])
    yy_list.append(y1)
    yy_list.append(y2)
    #図心を画像に記載する
    cv2.circle(frame1,(x1,y1),20,(0,0,255),-1)
    if taishi==len(idx_max_list)-1:
        cv2.circle(frame2,(x2,y2),20,(0,0,255),-1
        )
        cv2.imwrite(path3+'naturalframe_moment{:0=3}'.format(m)+".bmp",frame2)
    else:
    #画像を表示する
        cv2.imwrite(path3+'naturalframe_moment{:0=3}'.format(n)+".bmp",frame1)
    #plt.imshow(frame)
    #plt.show
    #print(y1,y2)
#----------------------------------------------------------------
#速度出すよ
    y_nat=y2-y1
    idx_y1=n
    idx_y2=m
    s_nat=(idx_y2-idx_y1)/30
    #print(idx_y1,idx_y2)
    v_nat=y_nat/s_nat
    #print('速度',v,'p/s')
#p/sをmm/sに変換してくよ
    u_nat=v_nat/mm
    uu_list.append(u_nat)
    ss_list.append(s_nat)
    print('自然上昇時上昇速度',u_nat,'mm/s')

error: OpenCV(4.0.1) C:\ci\opencv-suite_1573470242804\work\modules\imgproc\src\color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [7]:
#グラフ作るよ
#idx_max_list
#膨張バージョン
t1_plot=[]
for i in range(len(idx_max_list)):
    t1_idx=idx_max_list[i]/30
    #x_idx=idx_max_list[i]/30
    t1_plot.append(t1_idx)
#print(t1_plot)  
x1=t1_plot
#y座標の重複を取り除く 
y_plot1=[]
y_plot2=[]
#print(y_list)
for j in y_list:
    if not j in y_plot1:
        y_plot1.append(j)
#print(y_plot1)
for l in range(len(y_plot1)):
    #print(l)       
    y1_mm=y_plot1[l]/mm
    y_plot2.append(y1_mm)
#print(y_plot2)    
y1=y_plot2
#収縮バージョン
t2_plot=[]
for o in range(len(idx_min_list)):
    t2_idx=idx_min_list[o]/30
    #x_idx=idx_min_list[i]/30
    t2_plot.append(t2_idx)
#print(t2_plot)  
x2=t2_plot 
#y座標の重複を取り除く 
y_plot3=[]
y_plot4=[]
#print(yyy_list)
for p in yyy_list:
    if not p in y_plot3:
        y_plot3.append(p)
#print(y_plot3)
for q in range(len(y_plot3)):
    #print(q)       
    y2_mm=y_plot3[q]/mm
    y_plot4.append(y2_mm)
#print(y_plot4)
y2=y_plot4

#自然上昇バージョン
t3_plot=[]
for s in range(len(idx_min_list)):
    t3_idx=idx_min_list[s]/30
    #x_idx=idx_min_list[i]/30
    t3_plot.append(t3_idx)
#print(t2_plot)  
x3=t3_plot 
#y座標の重複を取り除く 
y_plot5=[]
y_plot6=[]
#print(y_list)
for t in yy_list:
    if not t in y_plot5:
        y_plot5.append(t)
#print(y_plot5)
for u in range(len(y_plot5)):
    #print(l)       
    y3_mm=y_plot5[u]/mm
    y_plot6.append(y3_mm)
y3=y_plot6
#print(x1,x2,x3)
#各々速度の平均値だす
mean_ex=round(sum(u_list)/float(len(u_list)),4)
mean_con=round(sum(uuu_list)/float(len(uuu_list)),4)
mean_nat=round(sum(uu_list)/float(len(uu_list)),4)
#print(mean_ex,mean_con,mean_nat)
fig, ax = plt.subplots()
ax.text(5,10,'expansionphase:'+str(mean_ex)+'[mm/s]',size=12,color='black')
ax.text(5,8,'contractionphase:'+str(mean_con)+'[mm/s]',size=12,color='black')
ax.text(5,6,'naturalphase:'+str(mean_nat)+'[mm/s]',size=12,color='black')

plt.plot(x1,y1,marker='.',color="red",label='expansion')
plt.plot(x2,y2,marker='.',color="blue",label='contraction')
plt.plot(x3,y3,marker='.',color="green",label='natural')
plt.title('Bubble rasing velocity phase')
plt.xlabel('t [s]')
plt.ylabel('y [mm]')
plt.xlim(t2_plot[0])
plt.ylim(0)
plt.show
plt.legend()
plt.savefig(path+'velocitygraph.png')



NameError: name 'yy_list' is not defined

In [6]:
#上昇速度だすよ
path4='C:\\Users\\flow\\Desktop\\2022.2.22ctab\\2022.2.22ctab4\\'
i=-10
j=0
u_list=[]
s_list=[]
y_list=[]
num_list=[]
for a in range(int(num/10)):
    i+=10
    j+=10
    #print(i,j)

    frame1=cv2.imread(path1+"kaitenpicture{:0=3}".format(i)+".bmp")
    frame2=cv2.imread(path1+"kaitenpicture{:0=3}".format(j)+".bmp")
    #print("save frame1picture{:0=3}".format(i)+".bmp")
    #print("save frame2picture{:0=3}".format(j)+".bmp")
    gray1=cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    gray2=cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    ret,th_otsu1 = cv2.threshold(gray1,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    ret,th_otsu2 = cv2.threshold(gray2,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    #モーメント関数で3次までのモーメントを取得
    mu1 = cv2.moments(th_otsu1)
    mu2 = cv2.moments(th_otsu2)
    #図心を出す
    x1,y1= int(mu1["m10"]/mu1["m00"]) , int(mu1["m01"]/mu1["m00"])
    x2,y2= int(mu2["m10"]/mu2["m00"]) , int(mu2["m01"]/mu2["m00"])
    y_list.append(y1)
    y_list.append(y2)
    #図心を画像に記載する
    cv2.circle(frame1,(x1,y1),20,(0,0,255),-1)
    if a==int(num/10):
            cv2.circle(frame2,(x2,y2),20,(0,0,255),-1)
            cv2.imwrite(path4+'_moment{:0=3}'.format(i)+".bmp",frame2)
    else:
    #画像を表示する
            cv2.imwrite(path4+'_moment{:0=3}'.format(j)+".bmp",frame1)
    #plt.imshow(frame)
    #plt.show
    #print(y1,y2)
    #----------------------------------------------------------------
    #速度出すよ
    y=y2-y1
    idx_y1=i
    idx_y2=j
    num_list.append(i)
    num_list.append(j)
    s=(idx_y2-idx_y1)/30
    #print(idx_y1,idx_y2)
    v=y/s
    #print('速度',v,'p/s')
    #p/sをmm/sに変換してくよ
    u=v/mm
    u_list.append(u)
    s_list.append(s)
    #print('上昇速度',u,'mm/s')
    #print(y_list)

#------------------------------------------------------------
#グラフ作るよ
#num_list
t1_list=[]
t2_list=[]
#リストの重複取り除く
for k in num_list:
    if not k in t1_list:
        t1_list.append(k)
#print(t1_list)        
for t in range(len(t1_list)):
    t=t1_list[t]/30
    t2_list.append(t)
#print(t2_list)
tt=t2_list
#y座標の重複を取り除く 
y_plot1=[]
y_plot2=[]
#print(y_list)
for j in y_list:
    if not j in y_plot1:
        y_plot1.append(j)
#print(y_plot1)
for l in range(len(y_plot1)):
    #print(l)       
    y1_mm=y_plot1[l]/mm
    y_plot2.append(y1_mm)
#print(t2_list,y_plot2)    
yy=y_plot2
#print(len(t2_list))
#print(len(y_plot2))

#各々速度の平均値だす
x=tt
y=yy
#print(tt,yy)
mean=round(sum(u_list)/float(len(u_list)),4)
fig, ax = plt.subplots()
ax.text(5,13,'velocity:'+str(mean)+'[mm/s]',size=15,color='black')
plt.plot(x,y,marker='.',color="red",label='rasing velocity')
plt.title('Bubble rasing velocity')
plt.xlabel('t [s]')
plt.ylabel('y [mm]')
plt.xlim(0)
plt.ylim(0)
plt.show
plt.legend()
plt.savefig(path4+'velocitygraph.png')

ZeroDivisionError: float division by zero